In [0]:
# predict why some batches of dog food are spoiling earlier than intended
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('rfc_consulting_proj').getOrCreate()
data = spark.read.csv("/FileStore/shared_uploads/sejal@ibm.com/dog_food.csv", inferSchema=True, header=True)

In [0]:
data.printSchema()

root
-- A: integer (nullable = true)
-- B: integer (nullable = true)
-- C: double (nullable = true)
-- D: integer (nullable = true)
-- Spoiled: double (nullable = true)

In [0]:
data.show()

+---+---+----+---+-------+
 A| B| C| D|Spoiled|
+---+---+----+---+-------+
 4| 2|12.0| 3| 1.0|
 5| 6|12.0| 7| 1.0|
 6| 2|13.0| 6| 1.0|
 4| 2|12.0| 1| 1.0|
 4| 2|12.0| 3| 1.0|
 10| 3|13.0| 9| 1.0|
 8| 5|14.0| 5| 1.0|
 5| 8|12.0| 8| 1.0|
 6| 5|12.0| 9| 1.0|
 3| 3|12.0| 1| 1.0|
 9| 8|11.0| 3| 1.0|
 1| 10|12.0| 3| 1.0|
 1| 5|13.0| 10| 1.0|
 2| 10|12.0| 6| 1.0|
 1| 10|11.0| 4| 1.0|
 5| 3|12.0| 2| 1.0|
 4| 9|11.0| 8| 1.0|
 5| 1|11.0| 1| 1.0|
 4| 9|12.0| 10| 1.0|
 5| 8|10.0| 9| 1.0|
+---+---+----+---+-------+
only showing top 20 rows

In [0]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=data.columns[:-1], outputCol='features')
output = assembler.transform(data)
output.show()

+---+---+----+---+-------+-------------------+
 A| B| C| D|Spoiled| features|
+---+---+----+---+-------+-------------------+
 4| 2|12.0| 3| 1.0| [4.0,2.0,12.0,3.0]|
 5| 6|12.0| 7| 1.0| [5.0,6.0,12.0,7.0]|
 6| 2|13.0| 6| 1.0| [6.0,2.0,13.0,6.0]|
 4| 2|12.0| 1| 1.0| [4.0,2.0,12.0,1.0]|
 4| 2|12.0| 3| 1.0| [4.0,2.0,12.0,3.0]|
 10| 3|13.0| 9| 1.0|[10.0,3.0,13.0,9.0]|
 8| 5|14.0| 5| 1.0| [8.0,5.0,14.0,5.0]|
 5| 8|12.0| 8| 1.0| [5.0,8.0,12.0,8.0]|
 6| 5|12.0| 9| 1.0| [6.0,5.0,12.0,9.0]|
 3| 3|12.0| 1| 1.0| [3.0,3.0,12.0,1.0]|
 9| 8|11.0| 3| 1.0| [9.0,8.0,11.0,3.0]|
 1| 10|12.0| 3| 1.0|[1.0,10.0,12.0,3.0]|
 1| 5|13.0| 10| 1.0|[1.0,5.0,13.0,10.0]|
 2| 10|12.0| 6| 1.0|[2.0,10.0,12.0,6.0]|
 1| 10|11.0| 4| 1.0|[1.0,10.0,11.0,4.0]|
 5| 3|12.0| 2| 1.0| [5.0,3.0,12.0,2.0]|
 4| 9|11.0| 8| 1.0| [4.0,9.0,11.0,8.0]|
 5| 1|11.0| 1| 1.0| [5.0,1.0,11.0,1.0]|
 4| 9|12.0| 10| 1.0|[4.0,9.0,12.0,10.0]|
 5| 8|10.0| 9| 1.0| [5.0,8.0,10.0,9.0]|
+---+---+----+---+-------+-------------------+
only showing top 20 rows

In [0]:
from pyspark.ml.classification import RandomForestClassifier
rfc = RandomForestClassifier(labelCol='Spoiled', featuresCol='features')
final_data = output.select('features', 'Spoiled')

In [0]:
rfc_model = rfc.fit(final_data)
rfc_model.featureImportances

Out[7]: SparseVector(4, {0: 0.0189, 1: 0.0141, 2: 0.9384, 3: 0.0287})

In [0]:
# the feature at index 2 is by far the most important feature: Chemical C